> 타이디 데이터를 만드는 방법에 대해 조금 더 자세히 알아보자!

## 1. 라이브러리 imports

In [1]:
import pandas as pd
import numpy as np

## 2. `pivot_table`, `groupby` + `aggregate`

> 대부분의 경우는 `pivot_table`로 해결이 되고, `groupby`로만 처리할 수 있는 것 몇 가지가 있긴 하다.

### **A. intro**
---




\- 개념 : 그룹화 -> 집계

`# 예제1`: 아래의 데이터프레임에서 (학과, 성별)로 count의 합계를 구하라

In [2]:
df=pd.read_csv("https://raw.githubusercontent.com/guebin/DV2022/master/posts/Simpson.csv",index_col=0,header=[0,1])\
.stack().stack().reset_index()\
.rename({'level_0':'department','level_1':'result','level_2':'gender',0:'count'},axis=1)
df

,department,result,gender,count
0,A,fail,female,19
1,A,fail,male,314
2,A,pass,female,89
3,A,pass,male,511
4,B,fail,female,7
5,B,fail,male,208
6,B,pass,female,18
7,B,pass,male,352
8,C,fail,female,391
9,C,fail,male,204


\- 행과 열에 데이터를 나눠서 표현하는 경우

In [3]:
df.pivot_table(index = 'department', columns = 'gender', values = 'count', aggfunc = np.sum)

gender,female,male
department,,
A,108,825
B,25,560
C,593,325
D,375,417
E,393,191
F,341,373


\- index에만 몰아주는 경우

In [4]:
df.pivot_table(index = ['department', 'gender'], values = 'count', aggfunc = np.sum)

count
department gender       
A          female    108
           male      825
B          female     25
           male      560
C          female    593
           male      325
D          female    375
           male      417
E          female    393
           male      191
F          female    341
           male      373

\- 예시에서 본 작업은 아래의 작업들로 세분화할 수 있다.

1. 그룹화(쿼리) : 하나의 DataFrame을 sub-dataframe으로 나누는 과정, 전체 자료를 **(학과, 성별)**로 묶어 총 12개의 sub-dataframe을 만든다.
1. 각각집계 : 나눠진 sub-dataframe에서 어떠한 계산을 각각 수행함, 나눠진 sub-dataframe에서 지원자 수의 합계를 각각 구함

\- 위와 같은 작업을 하려면 아래와 같은 요소들이 필요하다.

1. 그룹변수~(없는 용어임)~ : 그룹화를 위해 필요한 변수 : DataFrame을 sub-dataframe으로 나누는 역할. >> `index and columns`
> 범주형이거나 범주형으로 바꿀 수 있는 데이터
1. 집계변수~(이것도 없는 용어임)~ : 집계함수의 대상이 되는 변수 >> `values`
1. 집계함수 : 그룹화된 데이터프레임에 수행하는 계산을 정의하는 함수 >> `aggfunc`

### **B. `pivot_table`의 문법**
---




\- `pivot_table`의 문법

In [ ]:
df.pivot_table(
    index = 그룹변수
    columns = 그룹변수
    values = 집계변수
    aggfunc = 집계함수
)

\- index & columns에 그룹변수를 적절히 나눠서 입력한다.

\- 그룹변수 : string 혹은 list of string으로 전달한다.
* `'department'`, `['department']`, `['department', 'gender']`

\- 집계변수 : string 혹은 list of string으로 전달한다.

\- 집계함수 : 함수 자체를 전달하거나, 함수를 의미하는 문자열, 혹은 그것들의 리스트 형태로 전달한다.

`# 예시` : 집계함수를 전달하는 방법

**data**

In [6]:
df = pd.DataFrame({'category':['A']*5+['B']*5, 'value':np.concatenate([np.random.randn(5), np.random.randn(5)+10])})
df

,category,value
0,A,0.664567
1,A,0.373027
2,A,-0.262055
3,A,2.471452
4,A,3.162341
5,B,9.073412
6,B,8.459440
7,B,8.828235
8,B,9.413620
9,B,11.777137


**방법 1** - 함수 자체를 전달

In [7]:
df.pivot_table(index=['category'],values='value',aggfunc=np.sum) # 함수자체

,value
category,
A,6.409332
B,47.551844


In [9]:
f = np.sum
f([1,2,3])

6

> 요런 느낌으로 직접 함수를 지정해주는 게 제일 직관적이긴 하다. (사용자 정의 함수를 넣어도 된다! 확장성이 상당한 부분. ~애초에 numpy도 사람이 만든 거니까...~)

**방법 2** - 함수를 의미하는 문자열을 전달

In [ ]:
df.pivot_table(index=['category'],values='value',aggfunc='sum')  # 리스트

**방법 3** - 리스트를 전달

In [10]:
df.pivot_table(
    index=['category'],
    values='value',
    aggfunc=['sum',np.min,np.mean,np.max,'count']
)

,sum,amin,mean,amax,count
,value,value,value,value,value
category,,,,,
A,6.409332,-0.262055,1.281866,3.162341,5
B,47.551844,8.459440,9.510369,11.777137,5


> 단순히 집계함수들의 리스트를 넣어줬을 뿐인데? 각각의 집계치를 알아서 구해줬다.

### **C. `groupby` + `aggregate`의 문법**
---






\- `groupby` + `aggregate`

> `df.groupby(그룹변수).aggregate({집계변수:집계함수})`

그룹화를 한 후(`index, columns`), 무엇을 집계할 것인지 dictionary로 지정해준다.(`values, aggfunc`)

### 3. AIRLINE 자료로 연습

In [11]:
df=pd.read_csv('https://raw.githubusercontent.com/PacktPublishing/Pandas-Cookbook/master/data/flights.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58492 entries, 0 to 58491
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   MONTH      58492 non-null  int64  
 1   DAY        58492 non-null  int64  
 2   WEEKDAY    58492 non-null  int64  
 3   AIRLINE    58492 non-null  object 
 4   ORG_AIR    58492 non-null  object 
 5   DEST_AIR   58492 non-null  object 
 6   SCHED_DEP  58492 non-null  int64  
 7   DEP_DELAY  57659 non-null  float64
 8   AIR_TIME   57474 non-null  float64
 9   DIST       58492 non-null  int64  
 10  SCHED_ARR  58492 non-null  int64  
 11  ARR_DELAY  57474 non-null  float64
 12  DIVERTED   58492 non-null  int64  
 13  CANCELLED  58492 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 6.2+ MB


\- ChatGPT의 도움을 받아 해당 인포를 정리

1. MONTH: 비행이 이루어진 월을 나타냄. 1에서 12 사이의 값을 갖음.
1. DAY: 비행이 이루어진 일자를 나타냄. 월에 따라 1~28/29/30/31 사이의 값을 1. 가질 수 있음.
1. WEEKDAY: 비행이 이루어진 요일을 나타냄. 일반적으로 1(일요일)부터 7(토요일1)까지의 값을 갖음.
1. AIRLINE: 해당 항공편을 운영하는 항공사의 약어나 코드를 나타냄.
1. ORG_AIR: 비행기가 출발하는 공항의 약어나 코드를 나타냄.
1. DEST_AIR: 비행기가 도착하는 공항의 약어나 코드를 나타냄.
1. SCHED_DEP: 원래의 예정된 출발 시간을 나타냄. 시간은 일반적으로 HHMM 형식으로 표시될 수 있음.
1. DEP_DELAY: 출발 지연 시간을 나타냄. 음수 값은 조기 출발, 양수 값은 지연을 의미함.
1. AIR_TIME: 실제 공중에서 비행한 시간을 분 단위로 나타냄.
1. DIST: 비행 거리를 나타냄. 일반적으로 마일 또는 킬로미터로 표시됨.
1. SCHED_ARR: 원래의 예정된 도착 시간을 나타냄. SCHED_DEP와 같은 형식으로 표시될 수 있음.
1. ARR_DELAY: 도착 지연 시간을 나타냄. 음수는 조기 도착, 양수는 지연을 의미함.
1. DIVERTED: 항공편이 다른 곳으로 우회되었는지를 나타냄. 1은 우회, 0은 정상 경로를 의미함.
1. CANCELLED: 항공편이 취소되었는지 여부를 나타냄. 1은 취소, 0은 취소되지 않음을 의미함.

`# 예제1` : **항공사별**로 **도착지연시간**의 **평균**을 구하라.

\- 풀이 1 : `pivot_table`

In [13]:
df.pivot_table(index = 'AIRLINE', values = 'ARR_DELAY', aggfunc = np.mean)  ## 집계함수를 쓰지 않으면 평균이 디폴트

,ARR_DELAY
AIRLINE,
AA,5.542661
AS,-0.833333
B6,8.692593
DL,0.339691
EV,7.034580
F9,13.630651
HA,4.972973
MQ,6.860591
NK,18.436070


> 아마 분 단위일듯

\- 풀이 2 : `groupby().aggregate()`

In [15]:
df.groupby(by = 'AIRLINE').aggregate({'ARR_DELAY' : np.mean})

,ARR_DELAY
AIRLINE,
AA,5.542661
AS,-0.833333
B6,8.692593
DL,0.339691
EV,7.034580
F9,13.630651
HA,4.972973
MQ,6.860591
NK,18.436070


> 동일한 결과를 산출한다.

`# 예제2` : **항공사 별**로 **비행취소건수**의 **합계**를 구하라. 취소건수가 높은 항공사 순으로 **정렬**하라.

\- 풀이 1 : `.pivot_table()`을 이용

In [18]:
df.pivot_table(index = 'AIRLINE', values = 'CANCELLED', aggfunc = np.sum).sort_values(by = 'CANCELLED', ascending = False)

,CANCELLED
AIRLINE,
AA,154
MQ,152
EV,146
OO,142
UA,93
WN,93
DL,38
NK,25
US,21


\- 풀이 2 : `.groupby().aggregate()`를 이용

In [20]:
df.groupby(by = 'AIRLINE').agg({'CANCELLED' : np.sum}).sort_values(by = 'CANCELLED', ascending = False)  ## aggregate 대신 agg로 줄여도 됨

,CANCELLED
AIRLINE,
AA,154
MQ,152
EV,146
OO,142
UA,93
WN,93
DL,38
NK,25
US,21


`# 예제3` : **항공사별**로 **비행취소율**을 구하라. 비행취소율이 가장 높은 항공사 순으로 **정렬**하라

In [21]:
df.pivot_table(index = 'AIRLINE', values = 'CANCELLED', aggfunc = np.mean).sort_values('CANCELLED', ascending = False)
##df.groupby('AIRLINE').aggregate({'CANCELLED':'mean'}).sort_values('CANCELLED',ascending=False)

,CANCELLED
AIRLINE,
MQ,0.043791
EV,0.024923
OO,0.021554
AA,0.017303
NK,0.016491
US,0.013003
UA,0.011935
WN,0.011048
F9,0.007593


`# 예제4` : (항공사, 요일)별 비행취소건수와 비행취소율을 조사하라.

\- 풀이1 : `.pivot_table`이용

In [22]:
df.pivot_table(index = 'AIRLINE', columns = 'WEEKDAY', values = 'CANCELLED', aggfunc = [np.sum, np.mean])

sum                              mean                                \
WEEKDAY   1   2   3   4   5   6   7         1         2         3         4   
AIRLINE                                                                       
AA       41   9  16  20  18  21  29  0.032106  0.007341  0.011949  0.015004   
AS        0   0   0   0   0   0   0  0.000000  0.000000  0.000000  0.000000   
B6        0   1   0   0   0   0   0  0.000000  0.012658  0.000000  0.000000   
DL       10   8   8   3   3   4   2  0.006068  0.005208  0.005131  0.001940   
EV       30  21  20  22  11  16  26  0.034130  0.023918  0.022910  0.026895   
F9        3   1   0   0   1   2   3  0.016129  0.005376  0.000000  0.000000   
HA        0   0   0   0   0   0   0  0.000000  0.000000  0.000000  0.000000   
MQ       44  17  13  22  14  14  28  0.086785  0.032819  0.025145  0.039146   
NK        7   3   3   3   3   4   2  0.035354  0.013158  0.013953  0.013216   
OO       30  11  14  13  25  20  29  0.030581  0.011156  0.014478  0.013627   
UA       23  20   9   9  12   6  14  0.018914  0.017498  0.007799  0.007719   
US        6   4   1   1   2   1   6  0.026906  0.018018  0.004545  0.003953   
VX        1   0   0   2   1   0   2  0.007194  0.000000  0.000000  0.014184   
WN       16  25  18  10   7  10   7  0.012708  0.019562  0.014118  0.007911   

                                       
WEEKDAY         5         6         7  
AIRLINE                                
AA       0.014151  0.018667  0.021837  
AS       0.000000  0.000000  0.000000  
B6       0.000000  0.000000  0.000000  
DL       0.001982  0.003195  0.001294  
EV       0.013111  0.022504  0.030233  
F9       0.005155  0.011050  0.015625  
HA       0.000000  0.000000  0.000000  
MQ       0.028000  0.038356  0.055777  
NK       0.012821  0.019900  0.009390  
OO       0.026399  0.024125  0.031385  
UA       0.010195  0.006682  0.013487  
US       0.009050  0.004695  0.022814  
VX       0.006667  0.000000  0.014815  
WN       0.005828  0.010132  0.006066

> 보기 좀 불편함

In [23]:
df.pivot_table(index=['AIRLINE','WEEKDAY'],values='CANCELLED',aggfunc=[np.mean,sum])

mean       sum
                CANCELLED CANCELLED
AIRLINE WEEKDAY                    
AA      1        0.032106        41
        2        0.007341         9
        3        0.011949        16
        4        0.015004        20
        5        0.014151        18
...                   ...       ...
WN      3        0.014118        18
        4        0.007911        10
        5        0.005828         7
        6        0.010132        10
        7        0.006066         7

[98 rows x 2 columns]

> 잘 보이긴 하는데 다 보이지 않음. ~그거야 늘여서 보면 됨...~

\- 풀이 2 : `groupby().aggregate()`를 이용

In [24]:
df.groupby(by = ['AIRLINE', 'WEEKDAY']).aggregate({'CANCELLED' : [np.mean, np.sum]})

CANCELLED    
                     mean sum
AIRLINE WEEKDAY              
AA      1        0.032106  41
        2        0.007341   9
        3        0.011949  16
        4        0.015004  20
        5        0.014151  18
...                   ...  ..
WN      3        0.014118  18
        4        0.007911  10
        5        0.005828   7
        6        0.010132  10
        7        0.006066   7

[98 rows x 2 columns]

> 다만 이 친구는 unstack()을 해줘야 pivot_table()에서의 결과와 유사하게 활용할 수 있다.

`# 예제4` : (항공사, 요일)별로 `CANCELLED`는 평균과 합계를 구하고, `AIR_TIME`은 평균과 표준편차를 구하여라.

\- 풀이1 : `.pivot_table()`로는 불가능

\- 풀이2 : `.groupby()` + `.aggregate()`만 가능

In [26]:
df.groupby(by = ['AIRLINE', 'WEEKDAY']).aggregate({'CANCELLED' : [np.mean, np.sum], 'AIR_TIME' : [np.mean, np.std]})

CANCELLED        AIR_TIME           
                     mean sum        mean        std
AIRLINE WEEKDAY                                     
AA      1        0.032106  41  147.610569  73.442540
        2        0.007341   9  143.851852  73.211275
        3        0.011949  16  144.514005  73.340675
        4        0.015004  20  141.124618  69.220840
        5        0.014151  18  145.430966  76.711095
...                   ...  ..         ...        ...
WN      3        0.014118  18  104.219920  53.869040
        4        0.007911  10  107.200800  54.466218
        5        0.005828   7  107.893635  57.172695
        6        0.010132  10  109.247433  56.149388
        7        0.006066   7  107.602273  56.419207

[98 rows x 4 columns]

> `.pivot_table()`로도 구현할 수는 있을텐데 그럴 경우 데이터프레임 두개를 합쳐줘야 한다...

`# 예제5` : 운행구간(거리의 구간)을 그룹화하고, 운행구간 별 비행취소건수와 취소율을 구하여라.

In [32]:
pd.qcut(df.DIST, q = 4)   ## pd.cut()은 구간 별 개수가 균일하지 않다...

0          (391.0, 690.0]
1        (1199.0, 4502.0]
2          (391.0, 690.0]
3         (690.0, 1199.0]
4        (1199.0, 4502.0]
               ...       
58487    (1199.0, 4502.0]
58488      (391.0, 690.0]
58489     (66.999, 391.0]
58490     (690.0, 1199.0]
58491      (391.0, 690.0]
Name: DIST, Length: 58492, dtype: category
Categories (4, interval[float64, right]): [(66.999, 391.0] < (391.0, 690.0] < (690.0, 1199.0] <
                                           (1199.0, 4502.0]]

In [35]:
## 1행에서 운행구간을 그룹화, 2행에서 피벗테이블 생성
df.assign(DIST_CUT = pd.qcut(df.DIST, q = 4))\
.pivot_table(index = 'DIST_CUT', values = 'CANCELLED', aggfunc = [np.sum, np.mean])

,sum,mean
,CANCELLED,CANCELLED
DIST_CUT,,
"(66.999, 391.0]",334,0.022659
"(391.0, 690.0]",196,0.013503
"(690.0, 1199.0]",203,0.013637
"(1199.0, 4502.0]",148,0.010313


In [36]:
df.assign(DIST_CUT = pd.qcut(df.DIST, q = 4))\
.groupby(by = 'DIST_CUT').aggregate({'CANCELLED' : [np.sum, np.mean]})

CANCELLED          
                       sum      mean
DIST_CUT                            
(66.999, 391.0]        334  0.022659
(391.0, 690.0]         196  0.013503
(690.0, 1199.0]        203  0.013637
(1199.0, 4502.0]       148  0.010313

> 긴 구간일 수록 취소율이 낮은 것을 볼 수 있다.